<a href="https://colab.research.google.com/github/MAchinetyro/Cats-vs-Dogs/blob/main/Cats_vs_DOGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
import time
import copy
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
from PIL import Image

torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
!gdown --id 1Dvw0UpvItjig0JbnzbTgYKB-ibMrXdxk
!unzip -q dogs-vs-cats.zip
!unzip -q train.zip
!unzip -q test1.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Dvw0UpvItjig0JbnzbTgYKB-ibMrXdxk
To: /content/dogs-vs-cats.zip
100% 852M/852M [00:15<00:00, 56.1MB/s]


In [3]:

train_dir = './train'
test_dir = './test1'

train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

In [4]:
print(f'Number of images in {train_dir} is {len(train_files)}')
print(f'Number of images in {test_dir} is {len(test_files)}')

Number of images in ./train is 25000
Number of images in ./test1 is 12500


In [5]:
imgpath = os.path.join(train_dir, train_files[0])
print(imgpath)

./train/dog.5381.jpg


Creating our Transformation

In [6]:
transformations = transforms.Compose([transforms.Resize((60,60)),transforms.ToTensor()])

In [7]:
class Dataset():
    def __init__(self, filelist, filepath, transform = None):
        self.filelist = filelist
        self.filepath = filepath
        self.transform = transform

    def __len__(self):
        return int(len(self.filelist))

    def __getitem__(self, index):
        imgpath = os.path.join(self.filepath, self.filelist[index])
        img = Image.open(imgpath)

        if "dog" in imgpath:
            label = 1
        else:
            label = 0

        if self.transform is not None:
            img = self.transform(img)

        return (img, label)

In [8]:
# Create our train and test dataset objects
train = Dataset(train_files, train_dir, transformations)
val = Dataset(test_files, test_dir, transformations)

In [9]:
# Getting a data entry
train.__getitem__(0)

(tensor([[[0.0667, 0.1176, 0.1373,  ..., 0.4667, 0.3490, 0.3333],
          [0.0549, 0.1176, 0.1608,  ..., 0.4745, 0.3608, 0.3294],
          [0.0549, 0.1059, 0.1725,  ..., 0.4471, 0.3255, 0.2824],
          ...,
          [0.2902, 0.3059, 0.3098,  ..., 0.7529, 0.7059, 0.6863],
          [0.2706, 0.2745, 0.2745,  ..., 0.7176, 0.6863, 0.6980],
          [0.2706, 0.2549, 0.2627,  ..., 0.5608, 0.6667, 0.7373]],
 
         [[0.0902, 0.1020, 0.1255,  ..., 0.3569, 0.2549, 0.2392],
          [0.0471, 0.0863, 0.1412,  ..., 0.3765, 0.2824, 0.2471],
          [0.0431, 0.0745, 0.1373,  ..., 0.3608, 0.2549, 0.2078],
          ...,
          [0.1569, 0.1686, 0.1647,  ..., 0.7843, 0.7333, 0.7216],
          [0.1490, 0.1529, 0.1451,  ..., 0.7647, 0.7294, 0.7412],
          [0.1412, 0.1373, 0.1373,  ..., 0.6275, 0.7216, 0.7608]],
 
         [[0.0784, 0.0745, 0.0745,  ..., 0.2157, 0.1451, 0.1451],
          [0.0196, 0.0314, 0.0745,  ..., 0.2275, 0.1569, 0.1333],
          [0.0235, 0.0275, 0.0745,  ...,

In [10]:
# Getting the shape of a single image
print(val.__getitem__(0)[0].shape)
print(train.__getitem__(0)[0].shape)

torch.Size([3, 60, 60])
torch.Size([3, 60, 60])


Now we will Build our Model

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3,16,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
            )

        self.conv2 =   nn.Sequential(
            nn.Conv2d(16,32,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.conv3 =   nn.Sequential(
            nn.Conv2d(32,64,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.fc1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*5*5,256),
        nn.ReLU(),
        nn.Linear(256,128),
        nn.ReLU()
        )

        self.fc2 = nn.Sequential(
        nn.Linear(128,2),
        )

    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.softmax(x,dim = 1)